# Project Part 1

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)



## Introduction/Background

In this project I will attempting to train a model to predict news media bias. In order to do this I will be utilizing the dataset NLPCSS-20 from the Association for Computational Linguistics which can be found on Github. The dataset is a collection or corpus contrived of 6,964 news articles. The labels of how we describe the corpus are the following:
- title: title of news
- content: content
- source: source of the content
- allsides-bias: bias (left, center or right)
- misc: aurhor, date, etc
- adfontes_fair: how fair is the news article (bias, neutral or unknown)
- adfontes_political: how political bias skewed the article is (bias, neutral or unknown)
- event_id:event id

This is a text classification problem as how we are trying to classify combination of strings that demonstrate wheter a news content is politically bias or not. The combination of strings is the content itself, it is then labeled with how bias that content is. 


## Exploratory Data Analysis

First step is to analyze the data we have in order to figure out how our data looks like and what it is trying to tell us.

The dataset comes in a Json file so there are extra steps as to converting the data into a dataframe.

In [28]:
# import all of the python modules/packages you'll need here
import pandas as pd

As we can oberseve the json data set is not able to be formatted into a dataframe, this is becuase its format is unsupported by pandas and will have to be reformated.

In [29]:
# convert json file to dataframe
# path has original data, path2 is a small excerpt from og data w/ a fix to check if it is able to reformatt into df
path= "NLPCSS-20-main/data/released_data.json"
path2= "NLPCSS-20-main/data/example.json"

# test_df = pd.read_json(path, orient='records')
test_df2 = pd.read_json(path2, orient='records')

# print(test_df)
print (test_df2)

             source                                              title  \
0          Fox News  Trump blasts Howard Schultz, says ex-Starbucks...   
1         USA TODAY  Trump blasts former Starbucks CEO Howard Schul...   
2  Washington Times  Mick Mulvaney: Trump to secure border 'with or...   
3  Washington Times  Trump says 'we'll do the emergency' if border ...   

   event_id adfontes_fair adfontes_political    allsides_bias  \
0         0          bias               bias   From the Right   
1         0          bias            neutral  From the Center   
2         0          bias            neutral   From the Right   
3         0          bias            neutral   From the Right   

                                             content  \
0  Obama administration alum Roger Fisk and Repub...   
1  WASHINGTON – President Donald Trump took a swi...   
2  Acting White House chief of staff Mick Mulvane...   
3  President Trump repeated his vow Friday to dec...   

                      

In [30]:
#path to reformatted json file
path_data= 'NLPCSS-20-main/data/data.json'
#reformatt json file
with open(path, 'r') as rf:
    with open(path_data, 'w') as wf:
        #add [ at the beginning of the corpus
        wf.write('[\n')
        #add a comma at the end of each line
        lines= rf.read().splitlines()
        new_line= ',\n'.join(lines)
        wf.write(new_line)
        #add ] at the end of the corpus
        wf.write('\n]')
        


In [31]:
"""Convert reformatted dataset inot dataframe"""
df = pd.read_json(path_data, orient='records')
print(df)

                source                                              title  \
0             Fox News  Trump blasts Howard Schultz, says ex-Starbucks...   
1            USA TODAY  Trump blasts former Starbucks CEO Howard Schul...   
2     Washington Times  Mick Mulvaney: Trump to secure border 'with or...   
3     Washington Times  Trump says 'we'll do the emergency' if border ...   
4             BBC News  Trump backs down to end painful shutdown tempo...   
...                ...                                                ...   
7770          Politico                   Ann Romney's task: Humanize Mitt   
7771  Washington Times               'Mittigator' to make case for Romney   
7772          Fox News  Convention-bound Ryan slams Obama for presidin...   
7773          Politico                Ryan seeks comfort of Ron Paul fans   
7774    CNN (Web News)    Paul to supporters: 2012 the start of a new era   

      event_id adfontes_fair adfontes_political    allsides_bias  \
0      